<a href="https://colab.research.google.com/github/1xuan0621/git_practice/blob/master/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
!pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
from flask import Flask, request, abort, jsonify

import json

from flask_ngrok import run_with_ngrok

from linebot import LineBotApi, WebhookHandler

from linebot.exceptions import InvalidSignatureError

from linebot.models import *

import requests

In [96]:
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("cCC/sWeqpseuaq2I6V4y+p6+qeSleyTqT/porO8G/HqDpXwKN+0izUwvHmCxb9Uu9LrIVDJJU8HzNGVq6igSEGAbVDaOww6UNs7IMrI1V3o3ppPLFVhd4WSwOxJM2E17+nnjrgTfxG663Gs/Mq2xmQdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("36ddfc64d08a3da63dd98486c14f5314")

In [97]:
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    with open("/content/drive/MyDrive/ai-event.log", "a") as f:
        f.write(body)
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [98]:
# 追隨後發送消息

from linebot.models.events import FollowEvent

# from linebot.models import TextSendMessage, ImageSendMessage

@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):

  user_profile = line_bot_api.get_profile(event.source.user_id)

  with open("/content/drive/MyDrive/user.txt", "a") as myfile:
    myfile.write(json.dumps(vars(user_profile), sort_keys=True))
    myfile.write('\n')

  line_bot_api.reply_message(
      event.reply_token,
      [TextSendMessage('感謝您將旅遊機器人加入好友！請輸入@help獲得更多資訊，或直接使用圖文選單進行操作。')]
  )

In [99]:
# 圖文選單設置
menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 843
  },
  "selected": true,
  "name": "圖文選單 1",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 0,
        "y": 0,
        "width": 826,
        "height": 843
      },
      "action": {
        "type": "message",
        "text": "@景點"
      }
    },
    {
      "bounds": {
        "x": 842,
        "y": 0,
        "width": 811,
        "height": 843
      },
      "action": {
        "type": "message",
        "text": "@help"
      }
    },
    {
      "bounds": {
        "x": 1663,
        "y": 0,
        "width": 837,
        "height": 843
      },
      "action": {
        "type": "message",
        "text": "@美食"
      }
    }
  ]
}
"""

In [100]:
menuJson=json.loads(menuRawData)

lineRichMenuId = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(menuJson))
print(lineRichMenuId)


uploadImageFile=open("blockchain_demo.jpg",'rb')

setImageResponse = line_bot_api.set_rich_menu_image(lineRichMenuId,'image/jpeg',uploadImageFile)

print(setImageResponse)

linkResult=line_bot_api.link_rich_menu_to_user("U97f2bcd49832938af4e5939dc8784145", lineRichMenuId)

print(linkResult)

richmenu-56f32758f2b9fc2a415450d79aa8e04b
None
None


In [101]:
# 景點範本
carousel_template_message = TemplateSendMessage(
    alt_text='Carousel template',
    template=CarouselTemplate(
        columns=[
            CarouselColumn(
                thumbnail_image_url='https://img.onl/PO9aQy',
                title='蘭陽博物館',
                text='以「宜蘭是一座博物館」為理念，保存並維護宜蘭的自然與人文環境，透過研究典藏、展示教育等功能，是今日主要認識宜蘭的「窗口」',
                actions=[
                    URITemplateAction(
                        label='查看',
                        uri='https://www.lym.gov.tw/ch/index.html'
                    ),
                    PostbackTemplateAction(
                        label='接受任務',
                        text='@蘭陽博物館',
                        data='喜歡博物館'
                    ),
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://img.onl/LrBdy6',
                title='宜蘭火車站',
                text='原為復古的紅磚建築，而後配合周邊的丟丟噹森林和幾米公園，全面重新規劃改裝，吸引了眾多旅客駐足取景，為全台灣最可愛的火車站',
                actions=[
                    URITemplateAction(
                        label='查看',
                        uri='https://pfse64289.pixnet.net/blog/post/334217191-宜蘭火車站一日遊'
                    ),
                    PostbackTemplateAction(
                        label='接受任務',
                        text='@宜蘭火車站',
                        data='喜歡拍照'
                    ),
                ]
            ),CarouselColumn(
                thumbnail_image_url='https://img.onl/Qi7pUf',
                title='傳統藝術中心',
                text='傳統藝術，是民間生活藝術之美，先民薪傳文化之源。藉由各項研究、保存、傳襲及展演計畫的推動，建立臺灣的文化精神代表。術',
                actions=[
                    URITemplateAction(
                        label='查看',
                        uri='https://www.ncfta.gov.tw/'
                    ),
                    PostbackTemplateAction(
                        label='接受任務',
                        text='@傳統藝術中心',
                        data='喜歡博物館'
                    ),
                ]
            ),
        ]
    )
)

In [102]:
# 美食範本
carousel_template_message_food = TemplateSendMessage(
    alt_text='Carousel template',
    template=CarouselTemplate(
        columns=[
            CarouselColumn(
                thumbnail_image_url='https://img.onl/GQVKw8',
                title='正好鮮肉小籠包',
                text='很好吃就對了啦\n營業時間：10:00~15:00（週一公休）',
                actions=[
                    URITemplateAction(
                        label='打電話',
                        uri='tel://039325641'
                    ),
                    PostbackTemplateAction(
                        label='領取優惠券',
                        text='@小籠包優惠券',
                        data='poor'
                    ),
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://img.onl/rqN6gS',
                title='礁溪柯氏蔥油餅',
                text='我也沒吃過，反正很多人排隊。\n營業時間：09:00~18:00 (週三公休)',
                actions=[
                    URITemplateAction(
                        label='打電話',
                        uri='tel://0985873689'
                    ),
                    PostbackTemplateAction(
                        label='領取優惠券',
                        text='@蔥油餅優惠券',
                        data='poor'
                    ),
                ]
            ),CarouselColumn(
                thumbnail_image_url='https://img.onl/UmhpjP',
                title='蘭城晶英酒店紅樓櫻桃鴨',
                text='好吃，超級貴，自己好好想想。\n營業時間：11:00~15:00；17:00~21:00',
                actions=[
                    URITemplateAction(
                        label='打電話',
                        uri='tel://039101011'
                    ),
                    PostbackTemplateAction(
                        label='領取優惠券',
                        text='@櫻桃鴨優惠券',
                        data='rich'
                    ),
                ]
            ),
        ]
    )
)

In [103]:
# 按鍵模板
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "message",
            "label": "景點",
            "text": "@景點",
          },
          {
            "type": "message",
            "label": "美食",
            "text": "@美食",
          }
        ],
  )
)

In [104]:
## 點擊後，開啟相機
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)
## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)
## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我放棄:(", data="loser")
)

quickReplyList = QuickReply(
    items = [cameraQuickReplyButton, locationQRB, postbackQRB]
)
quick_reply_text_send_message = TextSendMessage(text='拍一張照片或是回報目前位置吧', quick_reply=quickReplyList)

In [105]:
# 設計字典，用戶輸入關鍵字輸出對應字
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)
template_message_dict = {
    "@help":buttons_template_message,
    "@景點":carousel_template_message,
    "@蘭陽博物館":quick_reply_text_send_message,
    "@宜蘭火車站":quick_reply_text_send_message,
    "@傳統藝術中心":quick_reply_text_send_message,
    "@美食":carousel_template_message_food,
    "@小籠包優惠券":TextSendMessage('目前無優惠券'),
    "@蔥油餅優惠券":TextSendMessage('目前無優惠券'),
    "@櫻桃鴨優惠券":TextSendMessage('目前無優惠券'),
}

In [106]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):

    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@help")
        )

In [107]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    elif (event.postback.data.find('喜歡博物館')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='感謝您接下蘭陽博物館的任務，請依照以下任務內容完成任務。'
                )
            )
    elif (event.postback.data.find('喜歡拍照') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='感謝您接下宜蘭火車站的任務，請依照以下任務內容完成任務。'
                )
            )
    elif (event.postback.data.find('喜歡傳統藝術') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='感謝您接下傳統藝術中心的任務，請依照以下任務內容完成任務。'
                )
            )
    else:
        pass

In [108]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://707e-35-225-135-142.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U231503724031c8bc9c1fb712a41cc6aa","events":[{"type":"message","message":{"type":"text","id":"16691912748651","text":"@景點"},"webhookEventId":"01GBMYQXF4ERXTRSF3FTWQB2V0","deliveryContext":{"isRedelivery":false},"timestamp":1661781931024,"source":{"type":"user","userId":"U97f2bcd49832938af4e5939dc8784145"},"replyToken":"c421b013c8594b1086c1bbf5615232ac","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Aug/2022 14:05:32] "POST / HTTP/1.1" 200 -


{"destination":"U231503724031c8bc9c1fb712a41cc6aa","events":[{"type":"message","message":{"type":"text","id":"16691913189562","text":"@help"},"webhookEventId":"01GBMYR2FQ5AQ5QJKD1JAGD3EN","deliveryContext":{"isRedelivery":false},"timestamp":1661781936156,"source":{"type":"user","userId":"U97f2bcd49832938af4e5939dc8784145"},"replyToken":"5ff35c2f7174489ab8046a2c8f8e86cf","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Aug/2022 14:05:37] "POST / HTTP/1.1" 200 -


{"destination":"U231503724031c8bc9c1fb712a41cc6aa","events":[{"type":"message","message":{"type":"text","id":"16691914128975","text":"@美食"},"webhookEventId":"01GBMYRCSQWYY911AVPAY5CGQK","deliveryContext":{"isRedelivery":false},"timestamp":1661781947082,"source":{"type":"user","userId":"U97f2bcd49832938af4e5939dc8784145"},"replyToken":"90a213ecbd56457eb2540b5db4e38114","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Aug/2022 14:05:47] "POST / HTTP/1.1" 200 -
